In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [4]:
## Inbuild Wikkipedia Tool
api_wrapper_wiki = WikipediaAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=250
)

tool_wiki = WikipediaQueryRun(
    api_wrapper=api_wrapper_wiki,
)

In [5]:
api_wrapper_arxiv = ArxivAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=250
)

tool_arxiv = ArxivQueryRun(
    api_wrapper=api_wrapper_arxiv,
)

In [6]:
tool_arxiv.name

'arxiv'

In [7]:
tools = [tool_wiki, tool_arxiv]

In [8]:
## Custom tool - RAG
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import faiss
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu", "token": "UdApfqIAOCtpiBtzcmyqSgMpMmTlZMLTPS"})
vectorstore = faiss.FAISS.from_documents(documents, embeddings)

retriever = vectorstore.as_retriever()

In [18]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FD813C8260>, search_kwargs={})

In [19]:
from langchain.tools.retriever import create_retriever_tool

retriver_tool = create_retriever_tool(
    retriever=retriever,
    name="langsmith-search",
    description="Search anything related to LangSmith documentation. Use this tool when you need to find information in the LangSmith documentation.",
)

retriver_tool.name

'langsmith-search'

In [20]:
tools = [tool_wiki, tool_arxiv, retriver_tool]

In [21]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\ajeet_singh\\Documents\\Ajeet\\Projects\\search_engine_langchain\\search_engine_langcahin\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search anything related to LangSmith documentation. Use this tool when you need to find information in the LangSmith documentation.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001FD81E4

In [22]:
## RUN these tools with Agents and LLMs

In [25]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [27]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="Llama3-8b-8192",
    api_key=groq_api_key,
    max_tokens=1000,
)

In [29]:
## prompt template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [30]:
## Agents to integrate tools with LLM
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(
    llm = llm,
    tools=tools,
    prompt=prompt,
)

In [31]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [32]:
## run agent
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [33]:
agent_executor.invoke({"input": "What is LangSmith?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get 

{'input': 'What is LangSmith?',
 'output': 'It seems that LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It provides tools for observability, evaluation, and prompt engineering to help developers create and optimize their applications.',
 'intermediate_steps': [(ToolAgentAction(tool='langsmith-search', tool_input={'query': 'LangSmith'}, log="\nInvoking: `langsmith-search` with `{'query': 'LangSmith'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'wbq1x5f7x', 'function': {'arguments': '{"query":"LangSmith"}', 'name': 'langsmith-search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand'}, id='run--9a3f04bb-6376-4722-bf3e-ab1c19c2163d', tool_calls=[{'name': 'langsmith-search', 'args': {'query': 'LangSmith'}, 'id': 'wbq1x5f7x', 'type': 'tool_call'}], usage_metadata={

In [34]:
agent_executor.invoke({"input": "What is transformer model?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Transformer model'}`


Page: Transformer (deep learning architecture)
Summary: In deep learning, transformer is an architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted i
Invoking: `wikipedia` with `{'query': 'Transformer model architecture'}`


Page: Transformer (deep learning architecture)
Summary: In deep learning, transformer is an architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted iThe transformer model is a type of deep learning architecture that is based on the multi-head attention mechanism. It was introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017. The transformer model is particularly well-suited for tasks such as machine translation, text summarization, and natural

{'input': 'What is transformer model?',
 'output': 'The transformer model is a type of deep learning architecture that is based on the multi-head attention mechanism. It was introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017. The transformer model is particularly well-suited for tasks such as machine translation, text summarization, and natural language processing.',
 'intermediate_steps': [(ToolAgentAction(tool='wikipedia', tool_input={'query': 'Transformer model'}, log="\nInvoking: `wikipedia` with `{'query': 'Transformer model'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'nva1yqh8g', 'function': {'arguments': '{"query":"Transformer model"}', 'name': 'wikipedia'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand'}, id='run--32f66e3a-80fc-4c16-833a-7ca3c35c447c', tool_calls=[{'nam

In [35]:
## Another way to make agent

In [36]:
from langchain.chat_models import init_chat_model
# from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [40]:
memory = MemorySaver()

In [50]:
model_2 = init_chat_model("Llama3-8b-8192", model_provider="groq", max_tokens = 1000, api_key=groq_api_key)

In [51]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model_2, tools, checkpointer=memory)

In [53]:
config = {"configurable": {"thread_id": "abc123"}}

In [60]:
input_message = {"role": "user", "content": "What is LangSmith? tell me pointwise adwantages. provide 5 points"}
response = agent_executor.invoke({"messages": [input_message]}, config=config)

# input_message = {"role": "user", "content": "What is langsmith?"}
# for step in agent_executor.stream(
#     {"messages": [input_message]}, config, stream_mode="values"
# ):
#     step["messages"][-1]

In [62]:
print(response["messages"][-1].content)

Here are 5 points about LangSmith, including its advantages:

**What is LangSmith?**
LangSmith is a platform for building production-grade LLM (Large Language Model) applications.

**Advantages:**

1. **Observability**: LangSmith provides LLM-native observability, allowing developers to get meaningful insights from their applications. This helps with debugging and identifying issues.
2. **Evaluation**: LangSmith offers high-quality evaluation datasets and metrics, making it easy to test and optimize LLM applications.
3. **Prompt Engineering**: LangSmith provides tools to help developers find the perfect prompt for their application, which is essential for LLM applications.
4. **Monitoring and Debugging**: LangSmith allows developers to monitor and debug their LLM applications, making it easier to identify and fix issues.
5. **Faster Development and Deployment**: LangSmith's features and tools enable faster development and deployment of LLM applications, which is critical for staying co